In [1]:
import pytz
import datetime
import pandas as pd
import collections
import re
from pymongo import MongoClient
import numpy as np

In [2]:
def removeSpecialCharacters(value):
    return re.sub('[^A-Za-z0-9]+', ' ', str(value))

def cleanDataColumns(data):
    trueCol = []
    for col in data.columns:
        trueCol.append(removeSpecialCharacters(col))
    data.columns = trueCol
    return data

def limpa_cpf(cpfString):
    cpfString = cpfString.replace('.','')
    cpfString = cpfString.replace('-','')
    return int(cpfString)    

def limpeza_de_cpfs(dados, coluna):
    dados[coluna] = dados.loc[:,coluna].apply(limpa_cpf)
    return dados
            
        
def coluna_para_datetime(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%d/%m/%Y')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

def string_to_datetime(datetime_string):
    datetime_string = datetime_string.split('/')
    datetime_value = datetime.datetime(
        int(datetime_string[2]),
        int(datetime_string[1]),
        int(datetime_string[0]))
    return date_to_Brazil_East_Utc(datetime_value)

def date_to_Brazil_East_Utc(datetime_value): # ele adianta o relogio em 3 horas
    if str(datetime_value) == 'NaT':
        datetime_value = datetime.datetime(1500,1,1) # minha data nulla e 01/01/1500
    local = pytz.timezone("Brazil/East")
    local_dt = local.localize(datetime_value, is_dst=True) # esse is_dst e pra ambiguidades, deve ser horario de verao aqui
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt.strftime ("%Y-%m-%d %H:%M:%S")

def float_latin_nan_e_zero(entry):
    if _isNan(entry):
        return 0.0e0
    else:
        return floatLatin(entry)
    

def floatLatin(entry):
    if _isNan(entry):
        return np.nan
    entry = str(entry).replace('.','')
    entry = str(entry).replace(',','.')
    return float(entry)

def _isNan(value):
    if isinstance(value, str):
        return value == 'nan'
    else:
        return np.isnan(value)


# TRANSFORMACOES ESPECIFICAS DA TABELA ACORDOS

def transforma_Assessoria(valor):
    if str(valor) == 'nan':
        return 'SEMEAR'
    else:
        return str(valor)

def transforma_Cobrador(valor):
    if 'Semear_Renego' in valor:
        return 'Renegociacao'
    elif 'Multiparcelas' in valor:
        return 'Multiparcelas'
    else:
        return 'NULO'

# TRANSFORMACOES ESPECIFICAS DA TABELA ACIONAMENTOS

def coluna_para_datetime_hora(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%d/%m/%Y %H:%M')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

# TRANSFORMACOES ESPECIFICAS DO RELATORIO 3040

def coluna_para_datetime_hora_seg(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%Y-%m-%d %H:%M:%S')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data


def string_para_datetime(datetime_string):
    return datetime.datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S')


# Leitura dos dados

In [3]:
# O ideal e que cada coluna tenha sua safra.

In [ ]:
dados = pd.read_excel('3040_01_2019.xlsx', encoding='latin-1', dtype=str) # demora uns 15min

In [5]:
dados = cleanDataColumns(dados)

In [18]:
dados = coluna_para_datetime_hora_seg(dados,'DatInicio')

NECESSARIO UMA PROTECAO AQUI PARA CORRIGIR PROBLEMAS NOS DADOS


In [11]:
colunas_desnecessarias = ['Data', 
                          'TipPessoa',
                          'Cliente',
                          'CodNatureza',
                          'TipControle',
                          'CodSistema',
                          'CodModalidade',
                          'Atividade',
                          'TotParcelas',
                          'RatCliente',
                          'RatOriginal',
                          'RatOperacao',
                          'PrazoContratado',
                          'Cosif',
                          'VlrPercInd',
                          'PorCliente',
                          'TipCessao',
                          'DatCessao',
                          'PctCedido',
                          'opccnjces',
                          'VlrNegociado',
                          'ContrCessao',
                          'SalCessao',
                          'opcidcrbc',
                          'CodModRenegociacao',
                          'DatRenegociacao',
                          'FatAnual',
                          'GrupoEconomico'
                         ]
dados = dados.drop(colunas_desnecessarias,axis=1)

In [12]:
dados = limpeza_de_cpfs(dados, 'CpfCgc')

In [13]:
dados.loc[:,'VlrSaldo'] = dados.loc[:,'VlrSaldo'].astype(float)

In [36]:
dados = coluna_para_datetime_hora_seg(dados,'DatProxParcela') # tem muitos null aqui que precisam de tratamento

In [42]:
dados.loc[:,'VlrProxParcela'] = dados.loc[:,'VlrProxParcela'].astype(float)

In [44]:
dados = coluna_para_datetime_hora_seg(dados,'DatVencimento')

In [45]:
dados = coluna_para_datetime_hora_seg(dados,'DatPrejuizo')

In [52]:
dados.loc[:,'CodProduto'].head(3)

0    3101
1    3104
2    3090
Name: CodProduto, dtype: object

In [ ]:
for i in range(10):
    Codigo_produto = int(dados.loc[i,'CodProduto'])

# Definicao da coluna liquidacoes

In [79]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.cobrancas
liquidacoes_mongo = db.liquidacoes

# Carga no banco de dados

In [85]:
# demora mais ou menos 1 hora essa carga aqui
liquidacao_ID = 0 
for i in dados.index:
    liquidacao_ID += 1
    Numero_do_contrato = dados.loc[i,'Contrato_sem_codigo']
    Codigo_renegociacao_contrato = dados.loc[i,'Contrato_codigo_renegociacao']
    Parcela = dados.loc[i, 'Parcela']
    Data_de_vencimento_da_parcela_liquidada = dados.loc[i, 'Vencimento']
    Data_de_liquidacao = dados.loc[i, 'Liquida o']
    Codigo_do_produto = dados.loc[i, 'produtos_codigos']
    Situacao_da_liquidacao = dados.loc[i,'Situa o']
    Numero_do_CPF = dados.loc[i,'CPF CNPJ']
    Dias_em_atraso_na_hora_da_liquidacao = dados.loc[i,'Dias']
    Codigo_da_forma_de_liquidacao = dados.loc[i,'codigo_da_forma_de_liquidacao']
    Estornado = dados.loc[i,'Estornado']
    Valor_recebido_na_liquidacao = dados.loc[i,'Recebido']
    dados_dict = {
        'Liquidacao_ID': liquidacao_ID,
        'Numero_do_contrato': int(Numero_do_contrato),
        'Data_da_liquidacao': Data_de_liquidacao,
        'Codigo_renegociacao_contrato': int(Codigo_renegociacao_contrato),
        'Parcela_liquidada': int(Parcela),
        'Data_de_vencimento_da_parcela_liquidada': Data_de_vencimento_da_parcela_liquidada,
        'Codigo_do_produto': int(Codigo_do_produto),
        'Situacao_da_liquidacao': str(Situacao_da_liquidacao), # poucos casos com situacao parcial aqui
        'Numero_do_CPF': int(Numero_do_CPF),
        'Dias_em_atraso_na_hora_da_liquidacao': int(Dias_em_atraso_na_hora_da_liquidacao),
        'Codigo_da_forma_de_liquidacao': int(Codigo_da_forma_de_liquidacao),
        'Estornado': str(Estornado),
        'Valor_recebido_na_liquidacao': Valor_recebido_na_liquidacao
    }

    liquidacoes_mongo.insert_one(dados_dict)

# Metadados que mapeiam os codigos dos produtos

In [107]:
informacoes_adicionais = db.informacoes_adicionais

In [108]:
#informacoes_adicionais.insert_one() Adicionar metadados aqui.

# Todos os registros para um dado CPF

In [88]:
import pprint
cursor = liquidacoes_mongo.find({'Numero_do_CPF': 60014075920})
for document in cursor:
    pprint.pprint(document)

{'Codigo_da_forma_de_liquidacao': 123,
 'Codigo_do_produto': 3101,
 'Codigo_renegociacao_contrato': 0,
 'Data_da_liquidacao': '2019-01-15 02:00:00',
 'Data_de_vencimento_da_parcela_liquidada': '2019-01-06 02:00:00',
 'Dias_em_atraso_na_hora_da_liquidacao': 9,
 'Estornado': 'NÃ£o',
 'Liquidacao_ID': 227478,
 'Numero_do_CPF': 60014075920,
 'Numero_do_contrato': 54245182,
 'Parcela_liquidada': 7,
 'Situacao_da_liquidacao': 'Liquidado',
 'Valor_recebido_na_liquidacao': 139.43,
 '_id': ObjectId('5d41d360d44318a2f4ee03a9')}
{'Codigo_da_forma_de_liquidacao': 123,
 'Codigo_do_produto': 3101,
 'Codigo_renegociacao_contrato': 0,
 'Data_da_liquidacao': '2019-02-12 02:00:00',
 'Data_de_vencimento_da_parcela_liquidada': '2019-02-06 02:00:00',
 'Dias_em_atraso_na_hora_da_liquidacao': 6,
 'Estornado': 'NÃ£o',
 'Liquidacao_ID': 573279,
 'Numero_do_CPF': 60014075920,
 'Numero_do_contrato': 54245182,
 'Parcela_liquidada': 8,
 'Situacao_da_liquidacao': 'Liquidado',
 'Valor_recebido_na_liquidacao': 137.35

# Todos os registros em um determinado intervalo de datas

In [ ]:
start = date_to_Brazil_East_Utc(datetime.datetime(2019, 6, 3))
end = date_to_Brazil_East_Utc(datetime.datetime(2019, 6, 4))
query_datas_limite = {'Data_da_liquidacao': {'$lt': end, '$gte': start}}
cursor = liquidacoes_mongo.find(query_datas_limite)
for document in cursor:
    print(document)